## Goal
* Gather sunset and sunrise times from an array of dates and locations and store results

https://sunrise-sunset.org/api

In [ ]:
# Import tools
from urllib.request import urlopen
from datetime import timedelta, date, datetime
from dateutil import tz
import json
import pandas as pd
import time
import os.path

## Build Date Array

In [ ]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Initialize date range start at 6/30 to compensate for UTC
start_date = date(2017, 6, 30)
end_date = date(2017, 11, 1)

dates = []

for single_date in daterange(start_date, end_date):
        dates.append(single_date.strftime("%Y-%m-%d"))

In [ ]:
print(dates[:4])
print(dates[-4:])

## Load Location Dictionary

In [ ]:
# Load station location csv into a DataFrame
stations = pd.read_csv('../output/mesoweb-station_meta-180919.csv')

In [ ]:
stations.info()

## Build URLs

In [ ]:
def build_url_df(stations, dates): 
    url_station = []
    # loop through stations in weather_monitoring-locations.csv
    for index, row in stations.iterrows():
        # loop through date range set above
        for date in dates:
            row_id = row['STID']
            lat = row['LATITUDE']
            lon = row['LONGITUDE']
            row_url = 'https://api.sunrise-sunset.org/json?lat=' + str(lat) + '&lng=' + str(lon) + '&date=' + date + '&formatted=0'
            url_station.append(dict({'station_id': row_id, 'url': row_url, 'date': date}))
    return(url_station)

In [ ]:
# Generate url list for api calls
url_batch = build_url_df(stations, dates)

In [ ]:
print('Number of urls: ' + str(len(url_batch)))
print('----\n', url_batch[-1:])

## Call API and store results

In [ ]:
def api_call(url):
    f = urlopen(url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    
    return parsed_json['results']

In [ ]:
def save_results(df_day, filepath):
    if os.path.isfile(filepath):
        with open(filepath, 'a') as f:
            df_day.to_csv(f, header=False, index=False)
    else:
        df_day.to_csv(filepath, index=False)

In [ ]:
%%time
successfull_calls = 0

# Loop over url_batch to call api and store results
for url in url_batch:
    # sleep to limit API call speed
    #time.sleep(3)
    
    print(url['station_id'] + '\n' + url['url'])
    result = api_call(url['url'])
    
    # include station_id
    result['station_id'] = url['station_id']
    
    # include date 
    result['date'] = url['date']
    
    df_day = pd.DataFrame.from_dict([result])

    save_results(df_day, '../output/sunrise_sunset-mesowest-utc.csv')
    
    successfull_calls += 1
    print('successfull_calls: ' + str(successfull_calls))

#### Additional Resources
[USNO](http://aa.usno.navy.mil/data/docs/RS_OneYear.php)